# <span style="color:green"> Import libraries<span style="color:green">

In [2]:
#Import all the required libraries
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn 
import random
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression
from sklearn.linear_model import Lasso, LassoCV, Ridge, RidgeCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import tensorflow as tf
from tensorflow import keras
from keras import layers
from tensorflow.keras.layers import Input, Dense, LeakyReLU, ReLU
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from sklearn.metrics import mean_absolute_error, mean_squared_error, explained_variance_score
from sklearn.feature_selection import f_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import pybiomart
from biomart import BiomartServer
from sklearn.feature_selection import RFECV
import xgboost as xgb
from sklearn.inspection import permutation_importance
import os

2025-01-06 10:01:32.804108: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-06 10:01:32.804221: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-06 10:01:32.910307: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-06 10:01:33.175267: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-06 10:01:35.012645: W tensorflow/compiler/tf2

# Define function for feature selection and association test

In [47]:
def process_file(file_path):
    # Load the data
    df = pd.read_table(file_path, delim_whitespace=True)
    print("Shape of file", df.shape)
    
    #X is Independent feature and y is dependent feature
    X= df.iloc[:, 6:]
    print("Shape of X", X.shape)
    y= df.PHENOTYPE
    print("Shape of y", y.shape)
    
    # Create and fit the Elasticnet model
    elastic_cv = linear_model.ElasticNetCV(l1_ratio=[0.5],alphas=[0.0033], cv=5, random_state= 2000)
    elastic_cv.fit(X, y)

    # Print the optimal alpha value
    print ("Optimal alpha:", elastic_cv.alpha_)
    print("Optimal l1_ratio: ", elastic_cv.l1_ratio_)

    # Indices of the SNPs with non-zero coefficients
    elastic_snps_ldl_arrange = np.flip(np.argsort(elastic_cv.coef_))[:5000]
    elastic_snps_ldl = np.where(elastic_cv.coef_)[0]

    # See how many SNPs have a non-zero coefficient
    print(len(elastic_snps_ldl), "selected SNPs")

    #Final SNPs
    E1_re= X.iloc[:,elastic_snps_ldl[:][:5000]]
    
    # Construct the output path
    output_dir = os.path.dirname(file_path)
    output_file = os.path.join(output_dir, "Elasticnet_SVR_5000")

    # Save the intermediate CSV in the same folder as the .raw file
    E1_re.to_csv(output_file, index=False)
    print(f"Results saved to: {output_file}")
    print("print shape of selected snps by Elastic net", E1_re.shape)
    
    Y= df.PHENOTYPE.values
    print("Y.shape",Y.shape)
    # Split Data
    x_train, x_test, Y_train, Y_test= train_test_split(E1_re, Y, test_size=0.3, random_state=2000)
    print(f"x_train={x_train.shape}, x_test={x_test.shape}, Y_train= {Y_train.shape}, Y_test= {Y_test.shape}")
    # Scale data
    scaler= StandardScaler()
    x_train= scaler.fit_transform(x_train)
    x_test= scaler.transform(x_test)
    
    
    # Association test using SVR
    svr = SVR(kernel = 'sigmoid' , C=100, epsilon= 0.01, gamma= 'scale', cache_size=800)
    svr.fit(x_train, Y_train)
    y_pred = svr.predict(x_test)
    r2_man = r2_score_manual(Y_test, y_pred)
    print("r2 is ",r2_man)
    mae = mean_absolute_error(Y_test, y_pred)
    variance = explained_variance_score(Y_test, y_pred)
    print("mean_absolute_error", mae)
    print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
    print("explained_variance_score", variance)
    
    return r2_man,mae,variance
    

## <span style="color:green"> r2_score_manual <span style="color:green"> 

In [48]:
def r2_score_manual(y_true, y_pred):
    # Convert inputs to NumPy arrays
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # Calculate the mean of the actual values
    y_mean = np.mean(y_true)
    
    # Residual sum of squares
    ss_res = np.sum((y_true - y_pred) ** 2)
    
    # Total sum of squares
    ss_tot = np.sum((y_true - y_mean) ** 2)
    
    # R² score
    r2 = 1 - (ss_res / ss_tot)
    return r2

In [49]:
# Define the root directory containing the subdirectories
root_dir = "/home/Vaishnavi/Simulation_rep_G2P"

# Initialize a list to store results
results = []

# Walk through subdirectories to find .raw files
for subdir, dirs, files in os.walk(root_dir):
    for file in files:
        if file.endswith(".raw"):
            file_path = os.path.join(subdir, file)
            print(f"Processing file: {file_path}")
            
            # Process the file and calculate metrics 
            try:
                r2, mae, variance = process_file(file_path)
                results.append({
                    "file": file,
                    "directory": subdir,
                    "r2_score": r2,
                    "mae": mae,
                    "explained_variance": variance
                })
            except Exception as e:
                print(f"Error processing {file_path}: {e}")


Processing file: /home/Vaishnavi/Simulation_rep_G2P/rep_1/GWAStutorialldlraw_1.raw
Shape of file (1124, 184515)
Shape of X (1124, 184509)
Shape of y (1124,)


/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.109e+03, tolerance: 1.809e+02
  model = cd_fast.enet_coordinate_descent(
/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.111e+03, tolerance: 1.872e+02
  model = cd_fast.enet_coordinate_descent(
/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Du

Optimal alpha: 0.0033
Optimal l1_ratio:  0.5
6644 selected SNPs
Results saved to: /home/Vaishnavi/Simulation_rep_G2P/rep_1/Elasticnet_SVR_5000
print shape of selected snps by Elastic net (1124, 5000)
Y.shape (1124,)
x_train=(786, 5000), x_test=(338, 5000), Y_train= (786,), Y_test= (338,)
r2 is  0.895129444359299
mean_absolute_error 12.210119342053224
mean_squared_error  226.45916189674676
explained_variance_score 0.8952438470572425
Processing file: /home/Vaishnavi/Simulation_rep_G2P/rep_2/GWAStutorialldlraw_2.raw
Shape of file (1124, 184515)
Shape of X (1124, 184509)
Shape of y (1124,)


/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.744e+02, tolerance: 1.539e+02
  model = cd_fast.enet_coordinate_descent(
/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.881e+02, tolerance: 1.535e+02
  model = cd_fast.enet_coordinate_descent(
/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Du

Optimal alpha: 0.0033
Optimal l1_ratio:  0.5
6178 selected SNPs
Results saved to: /home/Vaishnavi/Simulation_rep_G2P/rep_2/Elasticnet_SVR_5000
print shape of selected snps by Elastic net (1124, 5000)
Y.shape (1124,)
x_train=(786, 5000), x_test=(338, 5000), Y_train= (786,), Y_test= (338,)
r2 is  0.9169203621089936
mean_absolute_error 9.520777356910045
mean_squared_error  142.34422718878258
explained_variance_score 0.9171540892478006
Processing file: /home/Vaishnavi/Simulation_rep_G2P/rep_3/GWAStutorialldlraw_3.raw
Shape of file (1124, 184515)
Shape of X (1124, 184509)
Shape of y (1124,)


...

Optimal alpha: 0.0033
Optimal l1_ratio:  0.5
5823 selected SNPs
Results saved to: /home/Vaishnavi/Simulation_rep_G2P/rep_40/Elasticnet_SVR_5000
print shape of selected snps by Elastic net (1124, 5000)
Y.shape (1124,)
x_train=(786, 5000), x_test=(338, 5000), Y_train= (786,), Y_test= (338,)
r2 is  0.9279738314500846
mean_absolute_error 7.752781979030801
mean_squared_error  96.67430671694484
explained_variance_score 0.9282407000955825
Processing file: /home/Vaishnavi/Simulation_rep_G2P/rep_41/GWAStutorialldlraw.raw
Shape of file (1124, 184515)
Shape of X (1124, 184509)
Shape of y (1124,)


/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.162e+02, tolerance: 1.244e+02
  model = cd_fast.enet_coordinate_descent(
/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.572e+02, tolerance: 1.241e+02
  model = cd_fast.enet_coordinate_descent(
/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Du

Optimal alpha: 0.0033
Optimal l1_ratio:  0.5
5732 selected SNPs
Results saved to: /home/Vaishnavi/Simulation_rep_G2P/rep_41/Elasticnet_SVR_5000
print shape of selected snps by Elastic net (1124, 5000)
Y.shape (1124,)
x_train=(786, 5000), x_test=(338, 5000), Y_train= (786,), Y_test= (338,)
r2 is  0.9143581681099152
mean_absolute_error 8.691669992133365
mean_squared_error  116.18006969284879
explained_variance_score 0.9151196154638661
Processing file: /home/Vaishnavi/Simulation_rep_G2P/rep1-run-try/GWAStutorialldlraw.raw
Shape of file (1124, 184515)
Shape of X (1124, 184509)
Shape of y (1124,)


/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.233e+03, tolerance: 2.087e+02
  model = cd_fast.enet_coordinate_descent(
/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.265e+03, tolerance: 2.157e+02
  model = cd_fast.enet_coordinate_descent(
/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Du

Optimal alpha: 0.0033
Optimal l1_ratio:  0.5
6882 selected SNPs
Results saved to: /home/Vaishnavi/Simulation_rep_G2P/rep1-run-try/Elasticnet_SVR_5000
print shape of selected snps by Elastic net (1124, 5000)
Y.shape (1124,)
x_train=(786, 5000), x_test=(338, 5000), Y_train= (786,), Y_test= (338,)
r2 is  0.8821323668374238
mean_absolute_error 12.79903698921781
mean_squared_error  273.5286007826961
explained_variance_score 0.8826350694200411


In [50]:
# Save results to a CSV file
results_df = pd.DataFrame(results)
results_df.to_csv("r2_scores.csv", index=False)

print("Processing completed. R-square scores saved to r2_scores.csv.")

Processing completed. R-square scores saved to r2_scores.csv.


In [ ]:
# TRY

In [18]:
# Define the root directory containing the subdirectories
root_dir = "/home/Vaishnavi/Simulation_rep_G2P"

# Initialize a list to store results
results = []

# Walk through subdirectories to find .raw files
for subdir, dirs, files in os.walk(root_dir):
    for file in files:
        if file.endswith(".raw"):
            file_path = os.path.join(subdir, file)
            print(f"Processing file: {file_path}")
            


Processing file: /home/Vaishnavi/Simulation_rep_G2P/rep_1/GWAStutorialldlraw_1.raw
Processing file: /home/Vaishnavi/Simulation_rep_G2P/rep_2/GWAStutorialldlraw_2.raw
Processing file: /home/Vaishnavi/Simulation_rep_G2P/rep_3/GWAStutorialldlraw_3.raw
Processing file: /home/Vaishnavi/Simulation_rep_G2P/rep_4/GWAStutorialldlraw_4.raw
Processing file: /home/Vaishnavi/Simulation_rep_G2P/rep_5/GWAStutorialldlraw_5.raw
Processing file: /home/Vaishnavi/Simulation_rep_G2P/rep_6/GWAStutorialldlraw_6.raw
Processing file: /home/Vaishnavi/Simulation_rep_G2P/rep_7/GWAStutorialldlraw_7.raw
Processing file: /home/Vaishnavi/Simulation_rep_G2P/rep_8/GWAStutorialldlraw_8.raw
Processing file: /home/Vaishnavi/Simulation_rep_G2P/rep_9/GWAStutorialldlraw_9.raw
Processing file: /home/Vaishnavi/Simulation_rep_G2P/rep_10/GWAStutorialldlraw_10.raw
Processing file: /home/Vaishnavi/Simulation_rep_G2P/rep_11/GWAStutorialldlraw_11.raw
Processing file: /home/Vaishnavi/Simulation_rep_G2P/rep_12/GWAStutorialldlraw_12.ra